In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Data Loading

In [2]:
dfc = np.load('/neurospin/lbi/monkeyfmri/deepstim/workspace/2023_ASM_tembedding/gitproject/2023_asm_tembedding/data/DFCs/inputs_04-08.npy')
meta = pd.read_csv("/neurospin/lbi/monkeyfmri/deepstim/workspace/2023_ASM_tembedding/gitproject/2023_asm_tembedding/data/DFCs/meta_04-08.csv")

In [ ]:
#if original data
dfc = np.load('/neurospin/lbi/monkeyfmri/deepstim/database/ANESTHETIC_database/derivatives/reference_kmeans/inputs/inputs.npy')
meta = pd.read_csv("/neurospin/lbi/monkeyfmri/deepstim/database/ANESTHETIC_database/derivatives/reference_kmeans/inputs/metadata.tsv", sep="\t")

In [3]:
n_runs = len(meta)
n_wins = 461
dfc_train = dfc[meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])].reshape((-1, n_wins, 82, 82))
dfc_test = dfc[(meta["monkey"] == "jade")].reshape((-1, n_wins, 82, 82))
dfc_all = dfc.reshape((-1, n_wins, 82, 82))
conditions_train = meta["condition"][meta["monkey"].isin(['almira', 'khali', 'kimiko', 'rana'])]
conditions_test = meta["condition"][meta["monkey"].isin(['jade'])]

In [13]:
np.unique(conditions_train)

array(['awake', 'deep-propofol', 'ketamine', 'light-propofol',
       'sevoflurane2', 'sevoflurane4'], dtype=object)

In [23]:
dict_conditions = {"awake":0, "deep-propofol":1, "ketamine":2, "light-propofol":3, "sevoflurane2":4, "sevoflurane4":5}
dict_conditions_inverse = {0:"awake", 1:"deep-propofol", 2:"ketamine", 3:"light-propofol", 4:"sevoflurane2", 5:"sevoflurane4"}
y_train = np.concatenate([[dict_conditions[elem]]*461 for elem in conditions_train])
y_test = np.concatenate([[dict_conditions[elem]]*461 for elem in conditions_test])

## Linear Kernel

In [5]:
iu = np.triu_indices(82,k=1)
flat_dfc_train = dfc_train[:,:,iu[0],iu[1]].reshape(-1,3321)
flat_dfc_test = dfc_test[:,:,iu[0],iu[1]].reshape(-1,3321)

In [6]:
from sklearn.svm import SVC
X = flat_dfc_train
y = y_train
clf = SVC(gamma='auto')
clf.fit(X, y)

SVC(gamma='auto')

In [16]:
pred_train = clf.predict(X_train)

array([0, 0, 0, ..., 2, 5, 5])

In [ ]:
for i in range(6):
    print(dict_conditions_inverse[i], np.sum((pred_train == y_train) * (y_train == i)/len(y_train)))

In [ ]:
X_test = flat_dfc_test
pred_test = clf.predict(X_test)

In [26]:
dict_conditions_inverse[1], np.sum((pred_test == y_test) * (y_test == 1)/np.sum(y_test == 1))

('deep-propofol', 0.5032537960954446)

In [21]:
for i in range(6):
    try :
        print(dict_conditions_inverse[i], np.sum((pred_test == y_test) * (y_test == i)/np.sum(y_test == i)))
    except Exception as e:
        pass

awake 0.73234514340805
deep-propofol 0.5032537960954446
ketamine nan
light-propofol 0.5032537960954447
sevoflurane2 0.039913232104121475
sevoflurane4 0.4685466377440347


/tmp/ipykernel_77681/4016931580.py:3: RuntimeWarning: divide by zero encountered in divide
  print(dict_conditions_inverse[i], np.sum((pred_test == y_test) * (y_test == i)/np.sum(y_test == i)))
/tmp/ipykernel_77681/4016931580.py:3: RuntimeWarning: invalid value encountered in divide
  print(dict_conditions_inverse[i], np.sum((pred_test == y_test) * (y_test == i)/np.sum(y_test == i)))
